In [1]:
#Step (1): Read Data
import pandas as pd #Read data
import numpy as np  #Array

# Read in dataset (encoding = 'Big5'可顯示出繁體中文)
data_train = pd.read_excel("01train_fix.xlsx")
data_test = pd.read_excel("01test.xlsx")
data_out = pd.read_excel("01test.xlsx")
data_train.head() # Show data (看看是否讀取成功)

,手機螢幕溫度(平均),手機螢幕溫度(最大),手機背蓋溫度(平均),平均APP運作指標/小時,充電時間指標,手機電池電量,CPU跑分階級,手機外殼/包膜指標,外型平均曲率,上網頻率指標,通話頻率指標,新機/二手/老舊,內容容量比率,CPU效能等級,Underclocking
0,58.785714,63.833333,43.213333,231.000000,0.039474,61.255411,3,0.053652,-0.037180,6235.000000,6029.000000,0,0.999244,1,0
1,52.357143,57.857143,58.053333,92.090000,0.732759,91.393846,4,0.074205,0.155243,1750.000000,1422.000000,0,0.950294,2,1
2,30.674860,36.714286,36.661527,836.239303,0.229050,68.931202,3,0.121504,0.031717,2679.476981,2200.824592,0,1.000000,1,0
3,32.119048,41.571429,63.213333,588.800000,0.090909,54.455267,3,0.153716,0.009326,362.900000,0.000000,0,0.918970,1,1
4,41.214286,49.404762,52.486667,683.900000,0.072727,61.922799,3,0.097287,-0.030090,2777.000000,2001.000000,0,0.906411,1,0


In [2]:
data_train.drop(['外型平均曲率','手機外殼/包膜指標','上網頻率指標','手機螢幕溫度(平均)','平均APP運作指標/小時'], axis=1, inplace=True)
data_test.drop(['id','外型平均曲率','手機外殼/包膜指標','上網頻率指標','手機螢幕溫度(平均)','平均APP運作指標/小時'], axis=1, inplace=True)


In [3]:
data_train.describe()

,手機螢幕溫度(最大),手機背蓋溫度(平均),充電時間指標,手機電池電量,CPU跑分階級,通話頻率指標,新機/二手/老舊,內容容量比率,CPU效能等級,Underclocking
count,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000
mean,45.285916,51.776233,0.225773,75.191156,3.320000,2207.723020,0.297436,0.859740,1.320000,0.428462
std,15.545521,14.260686,0.232118,14.006018,0.684837,1810.477691,0.572736,0.184776,0.684837,0.494919
min,8.047619,2.562656,0.000000,47.889610,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,34.863095,43.508881,0.030190,62.766053,3.000000,579.800000,0.000000,0.802988,1.000000,0.000000
50%,47.011905,53.053333,0.143544,76.686508,3.000000,1898.000000,0.000000,0.937162,1.000000,0.000000
75%,57.339286,60.635000,0.371429,86.817917,3.000000,3454.268332,1.000000,0.987234,1.000000,1.000000
max,79.547619,118.066667,0.966667,100.000000,6.000000,8129.000000,5.000000,1.000000,4.000000,1.000000


In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler  #建構standardization的物件

#name_list_outlier = ['手機螢幕溫度(平均)','手機螢幕溫度(最大)','手機背蓋溫度(平均)','平均APP運作指標/小時','充電時間指標',
                #'手機電池電量','手機外殼/包膜指標','外型平均曲率','上網頻率指標',
                #'通話頻率指標','新機/二手/老舊','內容容量比率','CPU效能等級','CPU跑分階級']

#name_list_outlier = ['手機螢幕溫度(最大)' ,'手機背蓋溫度(平均)','充電時間指標','手機電池電量','通話頻率指標',
                        #'新機/二手/老舊','內容容量比率','CPU效能等級','CPU跑分階級']

name_list_outlier =[ '手機背蓋溫度(平均)',  '內容容量比率']
name_pre_outlier = ['Mobile Screen Temperature (Max)','Mobile Phone Back Cover Temperature (Average)','Average APP Operation Index/Hour','Mobile Phone Battery Level','Content Capacity Ratio']


#離群值處理
def  outlier_del(data_frame,name_list_outlier,n1=5,n2=5):
    for i, name in enumerate(name_list_outlier):
        print(name)
        print("Shape Of The Before Ouliers: ", data_frame[name].shape)
    
        # 计算IQR
        Q1 = np.percentile(data_frame[name], 25)
        Q3 = np.percentile(data_frame[name], 75)
        IQR = Q3 - Q1

        # 定义离群值范围
        lower_bound = Q1 - n1 * IQR
        upper_bound = Q3 + n2 * IQR

        # 过滤离群值
        filtered_entries = ((data_frame[name] >= lower_bound) & (data_frame[name] <= upper_bound))
        data_frame = data_frame[filtered_entries]
    
        print("Shape Of The After Ouliers: ",data_frame[name].shape)
        print('===================================================================================================================')

In [5]:
name_list_outlier =['手機螢幕溫度(最大)' ,'手機背蓋溫度(平均)','通話頻率指標','CPU效能等級','CPU跑分階級']

data_train=outlier_del(data_train,name_list_outlier)

手機螢幕溫度(最大)
Shape Of The Before Ouliers:  (3900,)
Shape Of The After Ouliers:  (3900,)
手機背蓋溫度(平均)
Shape Of The Before Ouliers:  (3900,)
Shape Of The After Ouliers:  (3900,)
通話頻率指標
Shape Of The Before Ouliers:  (3900,)
Shape Of The After Ouliers:  (3900,)
CPU效能等級
Shape Of The Before Ouliers:  (3900,)
Shape Of The After Ouliers:  (2999,)
CPU跑分階級
Shape Of The Before Ouliers:  (2999,)
Shape Of The After Ouliers:  (2999,)


In [6]:
'''
from sklearn.preprocessing import StandardScaler

# 定義需要標準化的欄位
columns_to_standardize = ['充電時間指標', '上網頻率指標', '通話頻率指標']

# 初始化標準化物件
standard_scaler  = StandardScaler()

# 將資料標準化
data_train[columns_to_standardize] = standard_scaler .fit_transform(data_train[columns_to_standardize])

# 顯示標準化後的資料
print(data_train.head())
'''

"\nfrom sklearn.preprocessing import StandardScaler\n\n# 定義需要標準化的欄位\ncolumns_to_standardize = ['充電時間指標', '上網頻率指標', '通話頻率指標']\n\n# 初始化標準化物件\nstandard_scaler  = StandardScaler()\n\n# 將資料標準化\ndata_train[columns_to_standardize] = standard_scaler .fit_transform(data_train[columns_to_standardize])\n\n# 顯示標準化後的資料\nprint(data_train.head())\n"

In [7]:
from sklearn.preprocessing import MinMaxScaler


columns_to_normalize = ['手機螢幕溫度(最大)' ,'手機背蓋溫度(平均)','通話頻率指標','CPU效能等級','CPU跑分階級','充電時間指標', '上網頻率指標', '通話頻率指標']

# 初始化標準化物件
min_max_scaler = MinMaxScaler()

# 將資料標準化
data_train[columns_to_normalize] = min_max_scaler.fit_transform(data_train[columns_to_normalize])

# 顯示標準化後的資料
print(data_train.head())

TypeError: 'NoneType' object is not subscriptable

In [ ]:
'''
from sklearn.preprocessing import StandardScaler

# 定義需要標準化的欄位
columns_to_standardize = ['充電時間指標', '上網頻率指標', '通話頻率指標']

# 初始化標準化物件
standard_scaler  = StandardScaler()

# 將資料標準化
data_test[columns_to_standardize] = standard_scaler .fit_transform(data_test[columns_to_standardize])
'''

"\nfrom sklearn.preprocessing import StandardScaler\n\n# 定義需要標準化的欄位\ncolumns_to_standardize = ['充電時間指標', '上網頻率指標', '通話頻率指標']\n\n# 初始化標準化物件\nstandard_scaler  = StandardScaler()\n\n# 將資料標準化\ndata_test[columns_to_standardize] = standard_scaler .fit_transform(data_test[columns_to_standardize])\n"

In [ ]:
from sklearn.preprocessing import MinMaxScaler


columns_to_normalize = ['手機螢幕溫度(最大)' ,'手機背蓋溫度(平均)','通話頻率指標','CPU效能等級','CPU跑分階級','充電時間指標', '上網頻率指標', '通話頻率指標']

# 初始化標準化物件
min_max_scaler = MinMaxScaler()

# 將資料標準化
data_test[columns_to_normalize] = min_max_scaler.fit_transform(data_test[columns_to_normalize])

In [ ]:
#Step (3): 用X儲存features, Y為labels(Underclocking)
y_data = data_train['Underclocking'].to_numpy()
feature_ex =['手機螢幕溫度(最大)' ,'手機背蓋溫度(平均)','充電時間指標','手機電池電量','通話頻率指標','新機/二手/老舊','內容容量比率','CPU效能等級','CPU跑分階級']
x_data = data_train[feature_ex].to_numpy()


In [ ]:
#Step (4): 把dataset分成訓練跟驗證
from sklearn.model_selection import train_test_split
#test_size=0.3 為比例-> training:test=7:3
trainX, testX, trainY, testY = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBClassifier
import numpy as np 
# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=10000, 
                             learning_rate=  0.3,
                             max_depth=5,
                             gamma=4.499999999999998)
# 使用訓練資料訓練模型
xgboostModel.fit(trainX, trainY)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(testX)
print('訓練集: ',xgboostModel.score(trainX,trainY))
print('測試集: ',xgboostModel.score(testX,testY))

訓練集:  0.7589743589743589
測試集:  0.7307692307692307


In [ ]:
predicted_ans = xgboostModel.predict(data_test)
my_submission = pd.DataFrame({'id': data_out.id, 'Underclocking': predicted_ans})
my_submission.to_csv('submission.csv', index=False)


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate, KFold
import numpy as np 

# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=10000, 
                             learning_rate=0.3,
                             max_depth=5,
                             gamma=4.499999999999998)

# 定义 KFold 交叉验证对象
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 使用 cross_validate 进行交叉验证
cv_results = cross_validate(xgboostModel, x_data, y_data, cv=kfold, scoring='accuracy', return_train_score=True)

# 输出每个 fold 的训练得分和验证得分
for i in range(5):
    train_score = cv_results['train_score'][i]
    test_score = cv_results['test_score'][i]
    print(f"Fold {i+1} - Training Score: {train_score}, Validation Score: {test_score}")

# 输出平均训练得分和平均验证得分
mean_train_score = np.mean(cv_results['train_score'])
mean_test_score = np.mean(cv_results['test_score'])
print(f"\nMean Training Score: {mean_train_score}")
print(f"Mean Validation Score: {mean_test_score}")


Fold 1 - Training Score: 0.7589743589743589, Validation Score: 0.7307692307692307
Fold 2 - Training Score: 0.7467948717948718, Validation Score: 0.7038461538461539
Fold 3 - Training Score: 0.7548076923076923, Validation Score: 0.7166666666666667
Fold 4 - Training Score: 0.7467948717948718, Validation Score: 0.717948717948718
Fold 5 - Training Score: 0.7618589743589743, Validation Score: 0.6871794871794872

Mean Training Score: 0.7538461538461538
Mean Validation Score: 0.7112820512820512
